In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/config.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/README.md
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/tokenizer.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/tokenizer_config.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/sentence_bert_config.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/config_sentence_transformers.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/model.safetensors
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/modules.json
/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/special_tokens_map.json
/kaggle/input/paraphras

In [2]:
import os, gc, time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss
import lightgbm as lgb


2025-10-28 04:26:12.619976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761625572.841845      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761625572.910569      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test  = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
sample_submission = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')

print("Train:", train.shape, " Test:", test.shape)
train.head(2)


Train: (57477, 9)  Test: (3, 4)


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0


In [4]:
print(train.columns)
print("\nMissing values:\n", train.isna().sum())
print("\nWinner counts:\n", train[['winner_model_a','winner_model_b','winner_tie']].sum())


Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')

Missing values:
 id                0
model_a           0
model_b           0
prompt            0
response_a        0
response_b        0
winner_model_a    0
winner_model_b    0
winner_tie        0
dtype: int64

Winner counts:
 winner_model_a    20064
winner_model_b    19652
winner_tie        17761
dtype: int64


In [5]:
for df in [train, test]:
    df['prompt'] = df['prompt'].fillna('')
    df['response_a'] = df['response_a'].fillna('')
    df['response_b'] = df['response_b'].fillna('')
    df['text_a'] = df['prompt'] + " " + df['response_a']
    df['text_b'] = df['prompt'] + " " + df['response_b']

train['label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)
train['label'].value_counts()


label
0    20064
1    19652
2    17761
Name: count, dtype: int64

In [6]:
def add_features(df):
    df['len_a'] = df['response_a'].str.len()
    df['len_b'] = df['response_b'].str.len()
    df['len_diff'] = df['len_a'] - df['len_b']
    df['words_a'] = df['response_a'].str.split().apply(len)
    df['words_b'] = df['response_b'].str.split().apply(len)
    df['words_diff'] = df['words_a'] - df['words_b']
    return df

train = add_features(train)
test = add_features(test)
print("Added text length features.")


Added text length features.


In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/kaggle/input/paraphrase-minilm-l3-v2-offline/paraphrase-MiniLM-L3-v2/')

emb_train_a = model.encode(train['text_a'].tolist(), show_progress_bar=True, batch_size=64)
emb_train_b = model.encode(train['text_b'].tolist(), show_progress_bar=True, batch_size=64)
emb_test_a  = model.encode(test['text_a'].tolist(), show_progress_bar=True, batch_size=64)
emb_test_b  = model.encode(test['text_b'].tolist(), show_progress_bar=True, batch_size=64)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Batches:   0%|          | 0/899 [00:00<?, ?it/s]

Batches:   0%|          | 0/899 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
def cosine_similarity(a, b):
    return np.sum(a*b, axis=1) / (np.linalg.norm(a, axis=1) * np.linalg.norm(b, axis=1) + 1e-12)

train['sim_ab'] = cosine_similarity(emb_train_a, emb_train_b)
train['emb_mean_abs_diff'] = np.mean(np.abs(emb_train_a - emb_train_b), axis=1)
train['emb_mean_diff'] = np.mean(emb_train_a - emb_train_b, axis=1)

test['sim_ab'] = cosine_similarity(emb_test_a, emb_test_b)
test['emb_mean_abs_diff'] = np.mean(np.abs(emb_test_a - emb_test_b), axis=1)
test['emb_mean_diff'] = np.mean(emb_test_a - emb_test_b, axis=1)


In [9]:
# STEP 5 - Create simple text-based features
train['len_a'] = train['text_a'].apply(len)
train['len_b'] = train['text_b'].apply(len)
train['len_diff'] = train['len_a'] - train['len_b']

train['words_a'] = train['text_a'].apply(lambda x: len(str(x).split()))
train['words_b'] = train['text_b'].apply(lambda x: len(str(x).split()))
train['words_diff'] = train['words_a'] - train['words_b']

test['len_a'] = test['text_a'].apply(len)
test['len_b'] = test['text_b'].apply(len)
test['len_diff'] = test['len_a'] - test['len_b']

test['words_a'] = test['text_a'].apply(lambda x: len(str(x).split()))
test['words_b'] = test['text_b'].apply(lambda x: len(str(x).split()))
test['words_diff'] = test['words_a'] - test['words_b']


In [10]:
feature_cols = [
    'len_a','len_b','len_diff','words_a','words_b','words_diff',
    'sim_ab','emb_mean_abs_diff','emb_mean_diff'
]

X = train[feature_cols].values
y = train['label'].values
X_test = test[feature_cols].values

print("Feature matrix shapes:", X.shape, X_test.shape)


Feature matrix shapes: (57477, 9) (3, 9)


In [11]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import numpy as np
from lightgbm import early_stopping, log_evaluation

params = {
    'objective': 'multiclass',
    'num_class': 3,
    'learning_rate': 0.05,
    'metric': 'multi_logloss',
    'verbose': -1,
    'random_state': 42
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros((len(train), 3))
test_preds = np.zeros((len(test), 3))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\nFOLD {fold+1}")
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]

    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dvalid = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtrain, dvalid],
        num_boost_round=2000,
        callbacks=[
            early_stopping(stopping_rounds=100),
            log_evaluation(period=100)
        ]
    )

    oof_preds[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test) / skf.n_splits

print("\nOOF Log Loss:", log_loss(y, oof_preds))



FOLD 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.01135	valid_1's multi_logloss: 1.0527
Early stopping, best iteration is:
[59]	training's multi_logloss: 1.02546	valid_1's multi_logloss: 1.05229

FOLD 2
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.01104	valid_1's multi_logloss: 1.05355
Early stopping, best iteration is:
[66]	training's multi_logloss: 1.02253	valid_1's multi_logloss: 1.0531

FOLD 3
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.01271	valid_1's multi_logloss: 1.04891
Early stopping, best iteration is:
[59]	training's multi_logloss: 1.02642	valid_1's multi_logloss: 1.04837

FOLD 4
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.01229	valid_1's multi_logloss: 1.05062
Early stopping, best iteration is:
[73]	training's multi_logloss: 1.02112	valid_1's multi_logloss: 1.05006

F

In [12]:
# Step 10: Create submission file for Kaggle
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_preds[:, 0],
    'winner_model_b': test_preds[:, 1],
    'winner_tie': test_preds[:, 2]
})

# normalize so probabilities sum to 1
submission[['winner_model_a', 'winner_model_b', 'winner_tie']] = submission[['winner_model_a', 'winner_model_b', 'winner_tie']].div(
    submission[['winner_model_a', 'winner_model_b', 'winner_tie']].sum(axis=1), axis=0
)

submission.to_csv('submission.csv', index=False)
print("✅ submission.csv file created successfully!")
submission.head()


✅ submission.csv file created successfully!


,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.268696,0.308125,0.423179
1,211333,0.474331,0.233266,0.292404
2,1233961,0.360803,0.314061,0.325136
